#  Calibration Image Master Creation notebook

This notebook calibrates Bias images and combines them into a Master Bias Image.

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
With the projects used above, any use of this notebook requires the citations of the Python packages and papers from which they are derived. 
Their citations are located here:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)



## How to Use

It's recommended you click "Run All Cells" to run this notebook, but you may optionally run each cell individually.

This notebook requires the location of your raw bias images as either a full location or as a relative path if you have the bias images in the folder that this notebook is located in.

Example:

Full location: C:\path\to\bias

Relative: path\to\bias



# Setup

All variables are set up in this cell. If this cell is not run, the program will not run.

In [1]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.stats import mad_std
from astropy.units import adu, second
from ccdproc import ImageFileCollection, combine, subtract_bias, subtract_dark
import matplotlib.pyplot as plt

import numpy as np

GIGABYTE = 1e9 # 1 billion bytes

data_directory = Path(".") # directory that notebooks are in.
reduced_path = data_directory.joinpath("reduced")
reduced_path.mkdir(exist_ok=True)


# Stage 1: Bias

During this stage the master Bias is made, it is integral to the rest of the process and must be done first.

In [2]:
biases = ImageFileCollection(data_directory, glob_include="*bias*.fit")

combined_bias = combine(biases.files, 
                        method="average", 
                        sigma_clip=True, 
                        sigma_clip_low_thresh=5, 
                        sigma_clip_high_thresh=5, 
                        sigma_clip_func=np.ma.median, 
                        sigma_clip_dev_func=mad_std,
                        mem_limit = GIGABYTE,
                        unit=adu)

combined_bias.meta['combined'] = True
combined_bias.meta['numcomb'] = len(biases.files)

combined_bias.write(reduced_path / "combined_bias.fit")

Set MJD-END to 59289.127362 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127362 from DATE-END'.
INFO:astropy:splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes.
Set MJD-END to 59289.127467 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127467 from DATE-END'.
Set MJD-END to 59289.127571 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127571 from DATE-END'.
Set MJD-END to 59289.127676 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127676 from DATE-END'.
Set MJD-END to 59289.127781 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127781 from DATE-END'.
Set MJD-END to 59289.127885 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127885 from DATE-END'.
Set MJD-END to 59289.127990 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.127990 from DATE-END'.


INFO: splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]



# Stage 2: Dark

Here the master Dark is made. This process requires the use of the master Bias, so Stage 1 must be run first

In [3]:
darks = ImageFileCollection(data_directory, glob_include="*dark*.fit")

# Calibration
calibrated_darks_path = data_directory.joinpath("bias_removed_darks")
calibrated_darks_path.mkdir(exist_ok=True)

for dark in darks.files:
    dark = Path(dark)
    dark_ccd = CCDData.read(dark, unit=adu)
    dark_ccd = subtract_bias(dark_ccd, combined_bias)
    dark_ccd.write(calibrated_darks_path.joinpath(dark))
    
calibrated_darks = ImageFileCollection(calibrated_darks_path, glob_include="*dark*.fit")
combined_dark = combine(calibrated_darks.files, 
                        method="average", 
                        sigma_clip=True, 
                        sigma_clip_low_thresh=5, 
                        sigma_clip_high_thresh=5, 
                        sigma_clip_func=np.ma.median, 
                        sigma_clip_dev_func=mad_std,
                        mem_limit = GIGABYTE,
                        unit=adu)

combined_dark.meta['combined'] = True
combined_dark.meta['numcomb'] = len(darks.files)

combined_dark.write(reduced_path / "combined_dark.fit")

Set MJD-END to 59289.107446 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.107446 from DATE-END'.
Set MJD-END to 59289.108597 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.108597 from DATE-END'.
Set MJD-END to 59289.109747 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.109747 from DATE-END'.
Set MJD-END to 59289.110897 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.110897 from DATE-END'.
Set MJD-END to 59289.112047 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.112047 from DATE-END'.
Set MJD-END to 59289.107446 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.107446 from DATE-END'.
INFO:astropy:splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes.
Set MJD-END to 59289.108597 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.108597 from DATE-END'.
Set MJD-END to 59289.109747 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.109747 from DATE-END'.
Set MJD-END to 59289.110897 from DATE-END'. [astropy.wcs.wc

INFO: splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]



# Stage 3: Flat

Here the master Flat is made. This process requires the use of the master Bias and master Dark, so Stage 1 and 2 must be run first

In [4]:
flats = ImageFileCollection(data_directory, glob_include="*flat*.fit")

# Calibration
calibrated_flats_path = data_directory.joinpath("bias_dark_removed_flats")
calibrated_flats_path.mkdir(exist_ok=True)

for flat in flats.files:
    flat = Path(flat)
    flat_ccd = CCDData.read(flat, unit=adu)
    flat_ccd = subtract_bias(flat_ccd, combined_bias)
    flat_ccd = subtract_dark(flat_ccd, combined_dark, exposure_time="EXPTIME", exposure_unit=second, scale=True)
    flat_ccd.write(calibrated_flats_path.joinpath(flat))
    
calibrated_flats = ImageFileCollection(calibrated_flats_path, glob_include="*flat*.fit")
combined_flat = combine(calibrated_flats.files, 
                        method="average", 
                        sigma_clip=True, 
                        sigma_clip_low_thresh=5, 
                        sigma_clip_high_thresh=5, 
                        sigma_clip_func=np.ma.median, 
                        sigma_clip_dev_func=mad_std,
                        mem_limit = GIGABYTE,
                        unit=adu)

combined_flat.meta['combined'] = True
combined_flat.meta['numcomb'] = len(flats.files)

combined_flat.write(reduced_path / "combined_flat.fit")

Set MJD-END to 59289.098337 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.098337 from DATE-END'.
Set MJD-END to 59289.098794 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.098794 from DATE-END'.
Set MJD-END to 59289.099249 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.099249 from DATE-END'.
Set MJD-END to 59289.099705 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.099705 from DATE-END'.
Set MJD-END to 59289.098337 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.098337 from DATE-END'.
INFO:astropy:splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes.
Set MJD-END to 59289.098794 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.098794 from DATE-END'.
Set MJD-END to 59289.099249 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.099249 from DATE-END'.
Set MJD-END to 59289.099705 from DATE-END'. [astropy.wcs.wcs]
Set MJD-END to 59289.099705 from DATE-END'.


INFO: splitting each image into 2 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]
